# The Land of Poets Challenge


For a digital humanities project you need to display Italian poets by filtering a csv table according to various criteria. This challenge will be only about querying with Polars, which is something you might find convenient to do during exams for quickly understanding datasets content (using Polars will always be optional, you will never be asked to perform complex modifications with it)

You are given a dataset taken from [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page), a project by the Wikimedia foundation which aims to store only machine-readable data, like numbers, strings, and so on interlinked with many references. Each entity in Wikidata has an identifier, for example Dante Alighieri is the entity [Q1067](https://www.wikidata.org/wiki/Q1067) and Florence is [Q2044](https://www.wikidata.org/wiki/Q2044)

Wikidata can be queried using the SPARQL language: the data was obtained with [this query](https://query.wikidata.org/#%23defaultView%3AMap%7B%22hide%22%3A%20%5B%22%3Fcoord%22%5D%7D%0ASELECT%20%3Fsubj%20%3FsubjLabel%20%3Fplace%20%3FplaceLabel%20%3Fcoord%20%3Fbirthyear%0AWHERE%20%7B%0A%20%20%20%3Fsubj%20wdt%3AP106%20wd%3AQ49757%20.%0A%20%20%20%3Fsubj%20wdt%3AP19%20%3Fplace%20.%0A%20%20%20%3Fplace%20wdt%3AP17%20wd%3AQ38%20.%0A%20%20%20%3Fplace%20wdt%3AP625%20%3Fcoord%20.%0A%20%20%20OPTIONAL%20%7B%20%3Fsubj%20wdt%3AP569%20%3Fdob%20%7D%0A%20%20%20BIND%28YEAR%28%3Fdob%29%20as%20%3Fbirthyear%29%0ASERVICE%20wikibase%3Alabel%20%7B%20%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22%20%7D%0A%7D) and downloaded in CSV format (among the many which can be chosen). Even if not necessary for the purposes of the exercise, you are invited to play a bit with the interface, like trying different visualizations (i.e. try select map in the middle-left corner) - or see [other examples](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [ ]:
import polars as pl

## Load the dataset

First load the dataset [italian-poets.csv](data/italian-poets.csv) in Polars dataframe `df` 

In [ ]:
df

subj,subjLabel,place,placeLabel,coord,birthyear,latitude,place_id,subj_id
str,str,str,str,str,i64,f64,str,str
"""http://www.wikidata.org/entity…","""Aemilius Macer""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",null,45.438611,"""Q2028""","""Q8797"""
"""http://www.wikidata.org/entity…","""Gaius Maecenas""","""http://www.wikidata.org/entity…","""Arezzo""","""Point(11.878055555 43.46305555…",null,43.463056,"""Q13378""","""Q8833"""
"""http://www.wikidata.org/entity…","""Michelangelo""","""http://www.wikidata.org/entity…","""Caprese Michelangelo""","""Point(11.985833333 43.64083333…",1475,43.640833,"""Q52069""","""Q5592"""
"""http://www.wikidata.org/entity…","""Horace""","""http://www.wikidata.org/entity…","""Venosa""","""Point(15.816666666 40.96666666…",-64,40.966667,"""Q52691""","""Q6197"""
"""http://www.wikidata.org/entity…","""Sallust""","""http://www.wikidata.org/entity…","""Amiternum""","""Point(13.305769 42.400776)""",-85,42.400776,"""Q177061""","""Q7170"""
…,…,…,…,…,…,…,…,…
"""http://www.wikidata.org/entity…","""Giovanni Marrasio""","""http://www.wikidata.org/entity…","""Noto""","""Point(15.083333333 36.88333333…",1405,36.883333,"""Q487174""","""Q99308713"""
"""http://www.wikidata.org/entity…","""Annemarie Innerebner""","""http://www.wikidata.org/entity…","""Bolzano""","""Point(11.35 46.5)""",1924,46.5,"""Q6526""","""Q100775377"""
"""http://www.wikidata.org/entity…","""Nesti Lyro Wollek""","""http://www.wikidata.org/entity…","""Florence""","""Point(11.254166666 43.77138888…",1875,43.771389,"""Q2044""","""Q100775410"""


## Tell me more

Show some info about the dataset

In [ ]:
info

statistic,subj,subjLabel,place,placeLabel,coord,birthyear
str,str,str,str,str,str,f64
"""count""","""3198""","""3198""","""3198""","""3198""","""3198""",3070.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",128.0
"""mean""",null,null,null,null,null,1724.418893
"""std""",null,null,null,null,null,373.394174
"""min""","""http://www.wikidata.org/entity…","""Abd ar-Rahman al-Itrabanishi""","""http://www.wikidata.org/entity…","""Aci Catena""","""Point(10.033333333 44.26666666…",-629.0
"""25%""",null,null,null,null,null,1647.0
"""50%""",null,null,null,null,null,1855.0
"""75%""",null,null,null,null,null,1922.0
"""max""","""http://www.wikidata.org/entity…","""Đuro Matijašević""","""http://www.wikidata.org/entity…","""ancient Syracuse""","""Point(9.983333333 45.316666666…",2000.0


## Getting in shape

Show the rows and the columns counts:

In [ ]:
print(f"rows: {nr_rows}")
print(f"columns: {nr_cols}")

rows: 3198
columns: 6


## 10 rows 

Display first 10 rows

In [ ]:
first_10

subj,subjLabel,place,placeLabel,coord,birthyear,latitude,place_id,subj_id
str,str,str,str,str,i64,f64,str,str
"""http://www.wikidata.org/entity…","""Aemilius Macer""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",null,45.438611,"""Q2028""","""Q8797"""
"""http://www.wikidata.org/entity…","""Gaius Maecenas""","""http://www.wikidata.org/entity…","""Arezzo""","""Point(11.878055555 43.46305555…",null,43.463056,"""Q13378""","""Q8833"""
"""http://www.wikidata.org/entity…","""Michelangelo""","""http://www.wikidata.org/entity…","""Caprese Michelangelo""","""Point(11.985833333 43.64083333…",1475,43.640833,"""Q52069""","""Q5592"""
"""http://www.wikidata.org/entity…","""Horace""","""http://www.wikidata.org/entity…","""Venosa""","""Point(15.816666666 40.96666666…",-64,40.966667,"""Q52691""","""Q6197"""
"""http://www.wikidata.org/entity…","""Sallust""","""http://www.wikidata.org/entity…","""Amiternum""","""Point(13.305769 42.400776)""",-85,42.400776,"""Q177061""","""Q7170"""
"""http://www.wikidata.org/entity…","""Ovid""","""http://www.wikidata.org/entity…","""Sulmona""","""Point(13.926198 42.048025)""",-42,42.048025,"""Q50157""","""Q7198"""
"""http://www.wikidata.org/entity…","""Grazia Deledda""","""http://www.wikidata.org/entity…","""Nuoro""","""Point(9.3280792 40.3200621)""",1871,40.320062,"""Q13649""","""Q7728"""
"""http://www.wikidata.org/entity…","""Bronzino""","""http://www.wikidata.org/entity…","""Florence""","""Point(11.254166666 43.77138888…",1503,43.771389,"""Q2044""","""Q7803"""
"""http://www.wikidata.org/entity…","""Sandra Lombardi""","""http://www.wikidata.org/entity…","""Rome""","""Point(12.482777777 41.89305555…",1946,41.893056,"""Q220""","""Q8796"""


## Born in Verona

Display all people born in Verona

In [ ]:
verona_df

subj,subjLabel,place,placeLabel,coord,birthyear
str,str,str,str,str,i64
"""http://www.wikidata.org/entity…","""Aemilius Macer""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",null
"""http://www.wikidata.org/entity…","""Catullus""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",-83
"""http://www.wikidata.org/entity…","""Girolamo Fracastoro""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1478
"""http://www.wikidata.org/entity…","""Guarino da Verona""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1374
"""http://www.wikidata.org/entity…","""Ippolito Pindemonte""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1753
…,…,…,…,…,…
"""http://www.wikidata.org/entity…","""Ilario Casarotti""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1772
"""http://www.wikidata.org/entity…","""Girolamo Orti Manara""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1769
"""http://www.wikidata.org/entity…","""Paolo Zazzaroni""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",null


## How many  people in Verona

Display how many people were born in Verona

In [ ]:
verona_count

37

## Python is everywhere

Show poets born in Catania in the year -500 

* mind the minus
* I swear we did not altered the dataset in any way :-)

In [ ]:
catania_minus_500

subj,subjLabel,place,placeLabel,coord,birthyear
str,str,str,str,str,i64
"""http://www.wikidata.org/entity…","""Python of Catana""","""http://www.wikidata.org/entity…","""Catania""","""Point(15.087269444 37.50266944…",-500


## Verona after 1500

Display all people born in Verona after the year 1500

In [ ]:
verona_after_1500

subj,subjLabel,place,placeLabel,coord,birthyear
str,str,str,str,str,i64
"""http://www.wikidata.org/entity…","""Ippolito Pindemonte""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1753
"""http://www.wikidata.org/entity…","""Aleardo Aleardi""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1812
"""http://www.wikidata.org/entity…","""Cristina Ali Farah""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1973
"""http://www.wikidata.org/entity…","""Francesco Scipione, marchese d…","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1675
"""http://www.wikidata.org/entity…","""Marco Antonio Zucchi""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1750
…,…,…,…,…,…
"""http://www.wikidata.org/entity…","""Giambattista Mutinelli""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1747
"""http://www.wikidata.org/entity…","""Pietro Caliari""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1841
"""http://www.wikidata.org/entity…","""Ilario Casarotti""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1772


## First Antonio

Display all people with Antonio as first name

In [ ]:
first_antonios

subj,subjLabel,place,placeLabel,coord,birthyear
str,str,str,str,str,i64
"""http://www.wikidata.org/entity…","""Antonio Bonfini""","""http://www.wikidata.org/entity…","""Ancona""","""Point(13.516666666 43.61666666…",1427
"""http://www.wikidata.org/entity…","""Antonio Bonfini""","""http://www.wikidata.org/entity…","""Patrignone""","""Point(13.60926 42.98027)""",1427
"""http://www.wikidata.org/entity…","""Antonio Tebaldeo""","""http://www.wikidata.org/entity…","""Ferrara""","""Point(11.619865 44.835297)""",1463
"""http://www.wikidata.org/entity…","""Antonio Fogazzaro""","""http://www.wikidata.org/entity…","""Vicenza""","""Point(11.55 45.55)""",1842
"""http://www.wikidata.org/entity…","""Antonio Ghislanzoni""","""http://www.wikidata.org/entity…","""Lecco""","""Point(9.4 45.85)""",1824
…,…,…,…,…,…
"""http://www.wikidata.org/entity…","""Antonio Bruni""","""http://www.wikidata.org/entity…","""Manduria""","""Point(17.634166666 40.40277777…",1593
"""http://www.wikidata.org/entity…","""Antonio Decio""","""http://www.wikidata.org/entity…","""Orte""","""Point(12.386111111 42.46027777…",1560
"""http://www.wikidata.org/entity…","""Antonio Rossetti""","""http://www.wikidata.org/entity…","""Vasto""","""Point(14.708219444 42.11158888…",1770


## Some Antonio

Display all people with Antonio as one of the names (so also include `'Paolo Antonio Rolli'`)

In [ ]:
some_antonio

subj,subjLabel,place,placeLabel,coord,birthyear
str,str,str,str,str,i64
"""http://www.wikidata.org/entity…","""Antonio Bonfini""","""http://www.wikidata.org/entity…","""Ancona""","""Point(13.516666666 43.61666666…",1427
"""http://www.wikidata.org/entity…","""Antonio Bonfini""","""http://www.wikidata.org/entity…","""Patrignone""","""Point(13.60926 42.98027)""",1427
"""http://www.wikidata.org/entity…","""Michelangelo Antonioni""","""http://www.wikidata.org/entity…","""Ferrara""","""Point(11.619865 44.835297)""",1912
"""http://www.wikidata.org/entity…","""Antonio Tebaldeo""","""http://www.wikidata.org/entity…","""Ferrara""","""Point(11.619865 44.835297)""",1463
"""http://www.wikidata.org/entity…","""Antonio Fogazzaro""","""http://www.wikidata.org/entity…","""Vicenza""","""Point(11.55 45.55)""",1842
…,…,…,…,…,…
"""http://www.wikidata.org/entity…","""Carlo Antonio Bertelli""","""http://www.wikidata.org/entity…","""Salò""","""Point(10.533333333 45.6)""",1637
"""http://www.wikidata.org/entity…","""Antonio Decio""","""http://www.wikidata.org/entity…","""Orte""","""Point(12.386111111 42.46027777…",1560
"""http://www.wikidata.org/entity…","""Antonio Rossetti""","""http://www.wikidata.org/entity…","""Vasto""","""Point(14.708219444 42.11158888…",1770


## Cesares during 1800

Display all people named Cesare who were born in the 1800s

In [ ]:
cesares_1800s

subj,subjLabel,place,placeLabel,coord,birthyear
str,str,str,str,str,i64
"""http://www.wikidata.org/entity…","""Cesare Meano""","""http://www.wikidata.org/entity…","""Turin""","""Point(7.7 45.066666666)""",1899
"""http://www.wikidata.org/entity…","""Cesare Betteloni""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",1808
"""http://www.wikidata.org/entity…","""Cesare De Titta""","""http://www.wikidata.org/entity…","""Sant'Eusanio del Sangro""","""Point(14.333333333 42.16666666…",1862
"""http://www.wikidata.org/entity…","""Cesare Pascarella""","""http://www.wikidata.org/entity…","""Rome""","""Point(12.482777777 41.89305555…",1858


## The old ones

Show poets in year of birth order, excluding null values from the result.

**HINT**: see [is_not_null()](https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.Expr.is_not_null.html)

In [ ]:
oldest_to_youngest

subj,subjLabel,place,placeLabel,coord,birthyear
str,str,str,str,str,i64
"""http://www.wikidata.org/entity…","""Stesichorus""","""http://www.wikidata.org/entity…","""Gioia Tauro""","""Point(15.9 38.433333333)""",-629
"""http://www.wikidata.org/entity…","""Ibycus""","""http://www.wikidata.org/entity…","""Reggio Calabria""","""Point(15.65 38.114438888)""",-600
"""http://www.wikidata.org/entity…","""Theognis of Megara""","""http://www.wikidata.org/entity…","""Megara Hyblaea""","""Point(15.18194444 37.20388889)""",-569
"""http://www.wikidata.org/entity…","""Parmenides""","""http://www.wikidata.org/entity…","""Velia""","""Point(15.154444444 40.15944444…",-514
"""http://www.wikidata.org/entity…","""Python of Catana""","""http://www.wikidata.org/entity…","""Catania""","""Point(15.087269444 37.50266944…",-500
…,…,…,…,…,…
"""http://www.wikidata.org/entity…","""Giacomo Panicucci""","""http://www.wikidata.org/entity…","""Piombino""","""Point(10.52212 42.934824)""",1985
"""http://www.wikidata.org/entity…","""Giovanni Bertoglio""","""http://www.wikidata.org/entity…","""Turin""","""Point(7.7 45.066666666)""",1986
"""http://www.wikidata.org/entity…","""Gio Evan""","""http://www.wikidata.org/entity…","""Molfetta""","""Point(16.6 41.2)""",1988


## Cities of poets

Find the 5 cities with most poets, sorted from most to least.

* use `group_by` and `sort` methods

In [ ]:
poets_cities.head()

placeLabel,len
str,u32
"""Rome""",198
"""Florence""",165
"""Milan""",121
"""Naples""",113
"""Venice""",94


## Most duplicated poets

Find first 8 duplicated poets

In [ ]:
most_duped

subjLabel,len
str,u32
"""Giambattista Andreini""",4
"""Sosiphanes""",4
"""Eliseo Calenzio""",4
"""Alojz Rebula""",4
"""Giovanni della Casa""",3
"""Aulus Gellius""",3
"""Tommaso Grossi""",3
"""Giuseppe Carpani""",3


## All duplicated poets

Print the number of all duplicated poets

In [ ]:
print(f"There are {dups} duplicated poets")

There are 118 duplicated poets


## Northern poets

Find all the poets born north of a given `town`. In other words, look for town latitude (the second coordinate in `coords`), print it,  and then filter the table.

- **DO NOT** put constants like `46.5` in your code!
- Start by extracting the latitudes as a new column.
- **NOTE**: `coord` column holds just simple strings!

In [ ]:
northern_poets

subj,subjLabel,place,placeLabel,coord,birthyear,latitude
str,str,str,str,str,i64,f64
"""http://www.wikidata.org/entity…","""Oswald von Wolkenstein""","""http://www.wikidata.org/entity…","""Schöneck Castle""","""Point(11.847977 46.820264)""",1377,46.820264
"""http://www.wikidata.org/entity…","""Simon Lemnius""","""http://www.wikidata.org/entity…","""Val Müstair""","""Point(10.39009 46.60566)""",1511,46.60566
"""http://www.wikidata.org/entity…","""Simon Lemnius""","""http://www.wikidata.org/entity…","""Val Müstair""","""Point(10.42 46.6)""",1511,46.6
"""http://www.wikidata.org/entity…","""Mary de Rachewiltz""","""http://www.wikidata.org/entity…","""Brixen""","""Point(11.65 46.716666666)""",1925,46.716667
"""http://www.wikidata.org/entity…","""Ignaz Vincenz Zingerle""","""http://www.wikidata.org/entity…","""Merano""","""Point(11.163888888 46.66888888…",1825,46.668889
…,…,…,…,…,…,…
"""http://www.wikidata.org/entity…","""Karl Ziegler""","""http://www.wikidata.org/entity…","""St. Martin in Passeier""","""Point(11.22727 46.78392)""",1812,46.78392
"""http://www.wikidata.org/entity…","""Fedele Demetz""","""http://www.wikidata.org/entity…","""Sëlva""","""Point(11.76038 46.55472)""",1850,46.55472
"""http://www.wikidata.org/entity…","""Roberta Dapunt""","""http://www.wikidata.org/entity…","""Val Badia""","""Point(11.89917 46.68361)""",1970,46.68361


## Papers please

Extract subject id (i.e. `Q8797`) and place id (i.e. `Q2028`) and MODIFY `df` by putting them into two new columns `subj_id` and `place_id`

In [ ]:
df

subj,subjLabel,place,placeLabel,coord,birthyear,latitude,place_id,subj_id
str,str,str,str,str,i64,f64,str,str
"""http://www.wikidata.org/entity…","""Aemilius Macer""","""http://www.wikidata.org/entity…","""Verona""","""Point(10.992777777 45.43861111…",null,45.438611,"""Q2028""","""Q8797"""
"""http://www.wikidata.org/entity…","""Gaius Maecenas""","""http://www.wikidata.org/entity…","""Arezzo""","""Point(11.878055555 43.46305555…",null,43.463056,"""Q13378""","""Q8833"""
"""http://www.wikidata.org/entity…","""Michelangelo""","""http://www.wikidata.org/entity…","""Caprese Michelangelo""","""Point(11.985833333 43.64083333…",1475,43.640833,"""Q52069""","""Q5592"""
"""http://www.wikidata.org/entity…","""Horace""","""http://www.wikidata.org/entity…","""Venosa""","""Point(15.816666666 40.96666666…",-64,40.966667,"""Q52691""","""Q6197"""
"""http://www.wikidata.org/entity…","""Sallust""","""http://www.wikidata.org/entity…","""Amiternum""","""Point(13.305769 42.400776)""",-85,42.400776,"""Q177061""","""Q7170"""
…,…,…,…,…,…,…,…,…
"""http://www.wikidata.org/entity…","""Giovanni Marrasio""","""http://www.wikidata.org/entity…","""Noto""","""Point(15.083333333 36.88333333…",1405,36.883333,"""Q487174""","""Q99308713"""
"""http://www.wikidata.org/entity…","""Annemarie Innerebner""","""http://www.wikidata.org/entity…","""Bolzano""","""Point(11.35 46.5)""",1924,46.5,"""Q6526""","""Q100775377"""
"""http://www.wikidata.org/entity…","""Nesti Lyro Wollek""","""http://www.wikidata.org/entity…","""Florence""","""Point(11.254166666 43.77138888…",1875,43.771389,"""Q2044""","""Q100775410"""


## Unknown poets 

Find all the ids of nameless poets and put them in **a python list**.

* **DO NOT** use loops
* **NOTE** a Series object  from the point of view of Python is just a sequence

In [ ]:
unknown_poets

['Q4360247',
 'Q14922292',
 'Q19130448',
 'Q21207901',
 'Q19984452',
 'Q21209119',
 'Q21282215',
 'Q23673492',
 'Q29049430',
 'Q29052339',
 'Q31763467',
 'Q28465822',
 'Q48809843',
 'Q27553577',
 'Q48811051',
 'Q48861610',
 'Q55441810',
 'Q47468550',
 'Q50327630',
 'Q50330028',
 'Q55897192',
 'Q65019765',
 'Q51845316',
 'Q60838260',
 'Q64433131',
 'Q71684946',
 'Q93338246',
 'Q59187521',
 'Q61136330',
 'Q61450547',
 'Q52107491',
 'Q61790603',
 'Q61791394',
 'Q61827513',
 'Q61895377',
 'Q59851133',
 'Q59851150',
 'Q62066746',
 'Q66736238',
 'Q66921487',
 'Q85421610',
 'Q61080035',
 'Q87068357',
 'Q64031897',
 'Q64167386',
 'Q64364409',
 'Q69818426',
 'Q64512266',
 'Q78162153',
 'Q78499894',
 'Q88264630',
 'Q89674973',
 'Q94998318',
 'Q94325725',
 'Q84138681',
 'Q98102965',
 'Q80705985',
 'Q81100287',
 'Q81738068',
 'Q83643534',
 'Q83808244',
 'Q96097742',
 'Q96245247',
 'Q96248786',
 'Q95485499',
 'Q99196947']

## Better unknown poets

Find all the ids, the birthplace and birthdate of nameless poets born after year 0, and put them in **a python list of dictionaries**.

* **DO NOT** use loops nor list comprehensions

In [ ]:
better_unknown_poets

[{'subj_id': 'Q4360247', 'birthyear': 1907, 'placeLabel': 'Rome'},
 {'subj_id': 'Q14922292', 'birthyear': 1991, 'placeLabel': 'Battipaglia'},
 {'subj_id': 'Q19130448', 'birthyear': 1492, 'placeLabel': 'Vicenza'},
 {'subj_id': 'Q21209119', 'birthyear': 1711, 'placeLabel': 'Giuliano Teatino'},
 {'subj_id': 'Q21282215', 'birthyear': 1590, 'placeLabel': 'Palermo'},
 {'subj_id': 'Q29052339', 'birthyear': 1733, 'placeLabel': 'Taranto'},
 {'subj_id': 'Q31763467', 'birthyear': 1755, 'placeLabel': 'Caltanissetta'},
 {'subj_id': 'Q28465822', 'birthyear': 1700, 'placeLabel': 'Orvieto'},
 {'subj_id': 'Q27553577', 'birthyear': 1250, 'placeLabel': 'Cavriana'},
 {'subj_id': 'Q48811051', 'birthyear': 1550, 'placeLabel': 'Roccabernarda'},
 {'subj_id': 'Q48861610', 'birthyear': 1452, 'placeLabel': 'Vittorio Veneto'},
 {'subj_id': 'Q55441810', 'birthyear': 1844, 'placeLabel': 'Vernio'},
 {'subj_id': 'Q47468550', 'birthyear': 1607, 'placeLabel': 'Florence'},
 {'subj_id': 'Q50327630', 'birthyear': 1700, 'p